In [6]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, roc_auc_score, cohen_kappa_score
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.feature_selection import VarianceThreshold
from sklearn.utils import shuffle

# Load the dataset
df = pd.read_csv("C:/RHEA/S7/Project/Code/Datasets/BRCA_gene_expression.csv", index_col="Unnamed: 0")
print(df.columns)

# Encode the target variable
le = LabelEncoder()
df["classes"] = le.fit_transform(df["classes"])  # Convert categorical labels to 0 and 1

# Define features and target
X = df.drop(columns=["classes"])
y = df["classes"]

# Split dataset into training and testing sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

print("Training set shape:", X_train.shape)
print("Testing set shape:", X_test.shape)

df = df.iloc[:,1:]
df.dropna()

model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)

# 4. Get feature importances

feature_importances = model.feature_importances_
feature_names = X.columns

# Create a DataFrame for importances
importance_df = pd.DataFrame({
    'Gene': feature_names,
    'Importance': feature_importances
})

# ---------------------------
# 5. Select Top 1000 Genes
# ---------------------------
top_genes_df = importance_df.sort_values(by='Importance', ascending=False).head(1000)





Index(['TSPAN6', 'TNMD', 'DPM1', 'SCYL3', 'FIRRM', 'FGR', 'CFH', 'FUCA2',
       'GCLC', 'NFYA',
       ...
       'C8orf44-SGK3', 'SNORA74C-2', 'ELOA3BP', 'NPBWR1', 'ELOA3DP', 'LNCDAT',
       'LOC124902537', 'RNF228', 'PANO1', 'classes'],
      dtype='object', length=31575)
Training set shape: (856, 31574)
Testing set shape: (368, 31574)


c:\RHEA\S7\Project\Code\Cancer-Classification\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [18:22:51] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [7]:

# Initialize and train the XGBoost classifier
xgb_clf = xgb.XGBClassifier(n_estimators=100, random_state=42, eval_metric="logloss")
xgb_clf.fit(X_train, y_train)

# Make predictions
y_pred = xgb_clf.predict(X_test)
y_pred_proba = xgb_clf.predict_proba(X_test)[:, 1]  # Probability estimates for AUC calculation

# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_proba)
kappa = cohen_kappa_score(y_test, y_pred)

# Print evaluation metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"AUC: {auc:.4f}")
print(f"Kappa: {kappa:.4f}")

Accuracy: 0.9837
Precision: 0.9375
AUC: 0.9948
Kappa: 0.9001


In [8]:
#XGB + RF
# Train XGBoost to get feature importances
xgb_selector = xgb.XGBClassifier(
    n_estimators=500, learning_rate=0.05, max_depth=6,
    subsample=0.8, colsample_bytree=0.8, random_state=42,
    use_label_encoder=False, eval_metric="logloss"
)

xgb_selector.fit(X_train, y_train)

# Get feature importances
feature_importances = pd.Series(xgb_selector.feature_importances_, index=X.columns)

# List of feature selection counts
feature_counts = [10, 50, 100, 500, 1000]

# Initialize a DataFrame to store results
results_df = pd.DataFrame(columns=["Features", "AUC", "Accuracy", "Precision", "Kappa"])

# Loop through different feature counts
for num_features in feature_counts:
    # Select top N features
    top_features = feature_importances.nlargest(num_features).index.tolist()

    # Filter dataset with selected features
    X_train_selected = X_train[top_features]
    X_test_selected = X_test[top_features]

    # Train Random Forest classifier
    rf_clf = RandomForestClassifier(n_estimators=500, max_depth=None, min_samples_split=2,
                                    random_state=42, n_jobs=-1)
    rf_clf.fit(X_train_selected, y_train)

    # Make predictions
    y_pred = rf_clf.predict(X_test_selected)
    y_pred_proba = rf_clf.predict_proba(X_test_selected)[:, 1]  # Probability estimates for AUC calculation

    # Evaluate model performance
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred_proba)
    kappa = cohen_kappa_score(y_test, y_pred)

    # Store results in DataFrame
    results_df = pd.concat([results_df, pd.DataFrame([[num_features, auc, accuracy, precision, kappa]],
                                                      columns=["Features", "AUC", "Accuracy", "Precision", "Kappa"])],
                                                      ignore_index=True)

# Display final results
print(results_df)

c:\RHEA\S7\Project\Code\Cancer-Classification\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [18:24:11] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\rheam\AppData\Local\Temp\ipykernel_30652\2429878928.py:45: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame([[num_features, auc, accuracy, precision, kappa]],


  Features       AUC  Accuracy  Precision     Kappa
0       10  0.995553  0.983696   0.911765  0.902783
1       50  0.996566  0.986413   0.939394  0.917901
2      100  0.996434  0.986413   0.939394  0.917901
3      500  0.996213  0.983696   0.937500  0.900145
4     1000  0.996302  0.983696   0.937500  0.900145
